In [0]:
# ============================================================================
# Credit Risk Analytics ETL Pipeline
# Production-Grade Banking Data Engineering Solution
# ============================================================================

from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import (
    col, when, lit, concat, upper, lower, substring, length,
    to_date, current_timestamp, year, month, dayofmonth,
    count, sum as spark_sum, avg, max as spark_max, min as spark_min,
    coalesce, row_number
)
from pyspark.sql.window import Window
from datetime import datetime
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

logger.info("Starting Credit Risk Analytics ETL Pipeline")

# ============================================================================
# STAGE 1: EXTRACT - Load raw data from sources
# ============================================================================

print("\n=== STAGE 1: EXTRACT ===")

# Create sample customer data for demonstration
customer_data = [
    (1, "John Doe", "john.doe@bank.com", "2015-01-15", "Active", "5000000"),
    (2, "Jane Smith", "jane.smith@bank.com", "2018-03-22", "Active", "7500000"),
    (3, "Bob Johnson", "bob.johnson@bank.com", "2019-06-10", "Inactive", "3000000"),
    (4, "Alice Williams", "alice.williams@bank.com", "2017-11-05", "Active", "9000000"),
    (5, "Charlie Brown", "charlie.brown@bank.com", "2016-09-18", "Active", "6500000"),
]

customer_df = spark.createDataFrame(
    customer_data,
    ["customer_id", "customer_name", "email", "account_open_date", "status", "annual_income"]
)

logger.info(f"Extracted {customer_df.count()} customer records")
print(f"Extracted {customer_df.count()} customer records")

# Create sample loan data
loan_data = [
    (1, 1, 500000, 9.5, "2023-01-15", "Active", 60, 8500, "Home Loan"),
    (2, 2, 750000, 8.75, "2023-02-20", "Active", 84, 10200, "Home Loan"),
    (3, 3, 300000, 12.5, "2023-03-10", "Defaulted", 36, 9800, "Personal Loan"),
    (4, 4, 900000, 7.99, "2023-04-05", "Active", 120, 8100, "Home Loan"),
    (5, 5, 650000, 9.99, "2023-05-12", "Active", 84, 9500, "Home Loan"),
]

loan_df = spark.createDataFrame(
    loan_data,
    ["loan_id", "customer_id", "loan_amount", "interest_rate", "disbursement_date", 
     "loan_status", "tenure_months", "emi_amount", "loan_type"]
)

logger.info(f"Extracted {loan_df.count()} loan records")
print(f"Extracted {loan_df.count()} loan records")

In [0]:
# ============================================================================
# STAGE 2: TRANSFORM - Data transformation and enrichment
# ============================================================================

print("\n=== STAGE 2: TRANSFORM ===")

# Data Quality Checks
logger.info("Performing data quality checks...")

# Convert data types
customer_df = customer_df.withColumn("account_open_date", to_date(col("account_open_date"), "yyyy-MM-dd")) \
                        .withColumn("annual_income", col("annual_income").cast("long"))

loan_df = loan_df.withColumn("disbursement_date", to_date(col("disbursement_date"), "yyyy-MM-dd"))

logger.info("Data type conversion completed")

# Join customer and loan data
merged_df = loan_df.join(customer_df, on="customer_id", how="left")

logger.info(f"Merged customer and loan data: {merged_df.count()} records")

# Calculate credit risk metrics
risk_df = merged_df.withColumn(
    "loan_to_income_ratio",
    col("loan_amount") / col("annual_income")
).withColumn(
    "annual_emi",
    col("emi_amount") * 12
).withColumn(
    "emi_to_income_ratio",
    col("annual_emi") / col("annual_income")
).withColumn(
    "risk_category",
    when(col("loan_status") == "Defaulted", "High")
    .when((col("emi_to_income_ratio") > 0.4) | (col("loan_to_income_ratio") > 0.8), "High")
    .when((col("emi_to_income_ratio") > 0.25) | (col("loan_to_income_ratio") > 0.5), "Medium")
    .otherwise("Low")
).withColumn(
    "processing_date",
    current_timestamp()
)

logger.info("Risk metrics calculated successfully")
print(f"Risk calculation completed for {risk_df.count()} records")

# Display sample transformed data
print("\nSample Transformed Data:")
risk_df.select(
    "customer_id", "loan_id", "loan_amount", "annual_income", 
    "loan_to_income_ratio", "risk_category"
).show()

In [0]:
# ============================================================================
# STAGE 3: LOAD - Write aggregated data to target systems
# ============================================================================

print("\n=== STAGE 3: LOAD ===")

# Create aggregation tables
print("\nCreating aggregation tables...")

# Portfolio summary by risk category
portfolio_summary = risk_df.groupBy("risk_category").agg(
    count("*").alias("num_loans"),
    spark_sum("loan_amount").alias("total_loan_amount"),
    avg("loan_to_income_ratio").alias("avg_loan_to_income_ratio"),
    spark_sum(col("loan_amount") * col("loan_to_income_ratio")).alias("risk_weighted_amount")
).withColumn("processing_date", current_timestamp())

logger.info("Portfolio summary created")
print("\nPortfolio Summary by Risk Category:")
portfolio_summary.show()

# Default risk analysis
default_analysis = risk_df.filter(col("loan_status") == "Defaulted").groupBy("risk_category").agg(
    count("*").alias("defaulted_loans"),
    spark_sum("loan_amount").alias("defaulted_amount"),
    avg("emi_to_income_ratio").alias("avg_emi_ratio")
)

logger.info("Default risk analysis completed")
print("\nDefault Risk Analysis:")
default_analysis.show()

# Customer level analytics
customer_analytics = risk_df.groupBy("customer_id", "customer_name", "status").agg(
    count("*").alias("num_loans"),
    spark_sum("loan_amount").alias("total_loan_exposure"),
    avg("loan_to_income_ratio").alias("avg_lti_ratio"),
    spark_max("risk_category").alias("max_risk_category")
).withColumn("processing_date", current_timestamp())

logger.info("Customer analytics created")
print("\nCustomer Analytics:")
customer_analytics.show()

# Quality Metrics
print("\n=== DATA QUALITY METRICS ===")
print(f"Total Records Processed: {risk_df.count()}")
print(f"High Risk Loans: {risk_df.filter(col('risk_category') == 'High').count()}")
print(f"Medium Risk Loans: {risk_df.filter(col('risk_category') == 'Medium').count()}")
print(f"Low Risk Loans: {risk_df.filter(col('risk_category') == 'Low').count()}")
print(f"Pipeline Status: SUCCESS")

logger.info("ETL Pipeline completed successfully")